In [1]:
import ee
import geemap

In [2]:
#Only have to do this once
#ee.Authenticate()

Enter verification code: 4/0QFKpd0zMOrrcIY3CqK4EEMBsqcP_D_Truv-9WcPv8GAipEf8OUlTG0

Successfully saved authorization token.


In [3]:
#Not sure this is necessary every time
#ee.Initialize()

In [13]:
#Create Map and center on Manauas EC Flux Tower coordinates
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 10)
#Create a point for EC Flux Tower
brma2 = ee.Geometry.Rectangle(-60.50, -2.85, -60.00, -2.35)
Map

Map(center=[-2.609097222, -60.20929722], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [14]:
#Create a point for EC Flux Tower
brma2 = ee.Geometry.Rectangle(-60.50, -2.85, -60.00, -2.35)

#brma2 = ee.Geometry.Point(-60.20929722, -2.609097222)

#Determine the area I want my square buffer to be
#areaM2 = 3.087969e9
#square = brma2.buffer(ee.Number(areaM2).sqrt().divide(2), 1).bounds()

#Add the polygon to the map
Map.addLayer(brma2, {}, 'Manauas Buffer')

In [29]:
#Load Sentinel-2 data and filter out cloud cover for tiles with less than 20% coverage
#Note: there is also a cloud masking function that I haven't tried. This was just easy.
sentinel2 = (ee.ImageCollection("COPERNICUS/S2") 
    .filterDate('2015-01-01', '2020-01-01') 
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
#    .filterBounds(brma2))
    .map(lambda image : image.clip(brma2)))

#Create some parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'max': 2000
}

Map.addLayer(sentinel2, vis_params, 'Raw Sentinel')

In [16]:
##Calculate the median to create one image; can only use an image with clip function
#s2_median = sentinel2.median()

##Clip median Sentinel-2 data to the square
#clipped = s2_median.clip(brma2)
#Map.addLayer(clipped, vis_params, 'Median Sentinel')

In [30]:
def addNDRE(image):
    return image.addBands(image.normalizedDifference(['B8', 'B5']).rename('NDRE'))


In [34]:
sentinel2 = sentinel2.map(addNDRE)
sentinel2_ndre= sentinel2.select('NDRE')
sentinel2_ndre_median = sentinel2_ndre.median()
#print(s2ndre, "list of available images")
#s2ndre_median = 


In [36]:
ndreParams={'min': 0.0,
            'max': 1.0,
            'palette' : ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
           }

Map.addLayer(sentinel2_ndre_median, ndreParams, 'NDRE Image')

In [20]:
#Calculate NDRE
#Select the least cloudy image
#Note: having to select only one image of the collection to calculate NDRE is REALLY annoying
#I can't find a way around it right now.
image = (ee.Image(
  sentinel2.filterBounds(brma2)
    .filterDate('2015-01-01', '2020-01-01')
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .first()))

#Set up NDRE calculation
nir = image.select('B8')
edge = image.select('B5')
ndre = nir.subtract(edge).divide(nir.add(edge)).rename('NDRE')
#ndreParams = {min: -1, max: 1, 'palette': ['blue', 'white', 'green']}
ndreParams={'min': 0.0,
            'max': 1.0,
            'palette' : ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
           }

Map.addLayer(ndre, ndreParams, 'NDRE Image')

addNDRE = (lambda image : image.addBands(ndre))

withNDRE = sentinel2.map(addNDRE)

In [253]:
#Make chart -- JK ui.Chart is only available in the code editor
#chart = ui.Chart.image.series({
#  imageCollection: withNDRE.select('NDRE'),
#  region: brma2,
#  reducer: ee.Reducer.first(),
#  scale: 30
#}).setOptions({title: 'NDRE over time'})
#print(chart)

In [254]:
#Put the EC Tower on top layer
Map.addLayer(brma2, {'color': 'FF0000', 'size': 50}, 'Manauas EC Flux Location')